In [26]:
# Helper libraries
import sys
sys.path.append('a3_framework')
import a3utils, a3tf_embed_viz, a3rnnlm
reload(a3utils)
reload(a3tf_embed_viz)
reload(a3rnnlm)

import util, rnnlm
reload(util)
reload(rnnlm)

import json, os, re, shutil, sys, time
import collections, itertools

from IPython.display import display, HTML
from random import randint

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("0.12"))

# utils.pretty_print_matrix uses Pandas. Configure float format here.
import pandas as pd
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

print "Done!"



Done!


In [6]:
samp = True
samp = '_samp' if samp else ''

In [7]:
# Load feature names
fnames = np.array(pd.read_pickle('../interim/028_preproc_heavy_shows_concat' + samp + '.p'))

# load concatenated descriptions
desc = np.array(pd.read_pickle('../interim/028_preproc_heavy_show_description_concat' + samp + '.p'))

print("Feature Names Shape:", fnames.shape)
print("Descriptions Shape:", desc.shape)
assert fnames.shape[0] == desc.shape[0]

('Feature Names Shape:', (1192, 2))
('Descriptions Shape:', (1192, 1))


In [8]:
reload(rnnlm)

TF_GRAPHDIR = "tf_graph"

# Clear old log directory.
shutil.rmtree(TF_GRAPHDIR, ignore_errors=True)

lm = rnnlm.RNNLM(V=10000, H=200, num_layers=2)
lm.BuildCoreGraph()
lm.BuildTrainGraph()
lm.BuildSamplerGraph()

summary_writer = tf.summary.FileWriter(TF_GRAPHDIR, lm.graph)
print "Done!"

Done!


In [87]:
# Load the dataset
reload(a3utils)
reload(a3rnnlm)

num_epochs = 5
num_elements = 5

#training parameters
training_params = dict(max_time=20, batch_size=50, learning_rate=0.5, num_epochs=num_epochs)

# Model parameters
vocab_size = 10000
model_params = dict(V=vocab_size, H=100, softmax_ns=200, num_layers=1)
desc_dict = {}
output = []
for i in range(0, desc.shape[0]):
    vocab, train_ids, test_ids = a3utils.load_corpus(desc[i][0], split=0.8, V=vocab_size, shuffle=42)
    final_h = a3rnnlm.execute_rnnlm(model_params, training_params, vocab, train_ids, test_ids)
    if final_h is not None:
        a = final_h.ravel()
        output.append(a)
        desc_dict[fnames[i,0]] = a
    if i >= (num_elements-1): break;

output_array = np.asarray(output)
print output_array.shape
print desc_dict.keys()

(91,)
Loaded 91 sentences (1514 tokens)
Training set: 72 sentences (1163 tokens)
Test set: 19 sentences (351 tokens)
[epoch 1] Starting epoch 1
[epoch 1] Completed in 0:00:00
[epoch 1] Train set: avg. loss: 8.976  (perplexity: 7914.86)
[epoch 1] Test set: avg. loss: 9.077  (perplexity: 8751.93)

[epoch 2] Starting epoch 2
[epoch 2] Completed in 0:00:00
[epoch 2] Train set: avg. loss: 8.528  (perplexity: 5056.49)
[epoch 2] Test set: avg. loss: 8.698  (perplexity: 5990.23)

[epoch 3] Starting epoch 3
[epoch 3] Completed in 0:00:00
[epoch 3] Train set: avg. loss: 8.135  (perplexity: 3411.50)
[epoch 3] Test set: avg. loss: 8.400  (perplexity: 4448.95)

[epoch 4] Starting epoch 4
[epoch 4] Completed in 0:00:00
[epoch 4] Train set: avg. loss: 7.689  (perplexity: 2183.60)
[epoch 4] Test set: avg. loss: 8.068  (perplexity: 3191.16)

[epoch 5] Starting epoch 5
[epoch 5] Completed in 0:00:00
[epoch 5] Train set: avg. loss: 7.498  (perplexity: 1804.47)
[epoch 5] Test set: avg. loss: 7.926  (perpl

/home/michaelrubin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


Loaded 688 sentences (11082 tokens)
Training set: 550 sentences (8992 tokens)
Test set: 138 sentences (2090 tokens)
[epoch 1] Starting epoch 1
[epoch 1] Completed in 0:00:02
[epoch 1] Train set: avg. loss: 7.640  (perplexity: 2079.20)
[epoch 1] Test set: avg. loss: 7.587  (perplexity: 1971.68)

[epoch 2] Starting epoch 2
[epoch 2] Completed in 0:00:02
[epoch 2] Train set: avg. loss: 7.277  (perplexity: 1446.24)
[epoch 2] Test set: avg. loss: 7.213  (perplexity: 1356.90)

[epoch 3] Starting epoch 3
[epoch 3] Completed in 0:00:02
[epoch 3] Train set: avg. loss: 7.124  (perplexity: 1240.86)
[epoch 3] Test set: avg. loss: 7.086  (perplexity: 1195.25)

[epoch 4] Starting epoch 4
[epoch 4] Completed in 0:00:02
[epoch 4] Train set: avg. loss: 6.929  (perplexity: 1021.14)
[epoch 4] Test set: avg. loss: 6.885  (perplexity: 977.19)

[epoch 5] Starting epoch 5
[epoch 5] Completed in 0:00:02
[epoch 5] Train set: avg. loss: 6.916  (perplexity: 1008.28)
[epoch 5] Test set: avg. loss: 6.857  (perplex

In [127]:
reload(util)
l = len(desc_dict.keys())-1
i = randint(0,l-1)

Dv = output_array
v = desc_dict[desc_dict.keys()[i]]
k=1
r = util.find_nn_cos( v, Dv, k=10)
print r[1]
print i, r[0],l

print "Selection:", desc_dict.keys()[i]
print '-'*70
count = 1
for a, b in zip(r[0], r[1]):
    print "%d. %f %s" %( count, b, desc_dict.keys()[a])
    count += 1

[ 1.          0.06955069  0.06752861 -0.02072901]
2 [3 2 1 0] 3
Selection: 2009 K-12 Online Conference Audio Podcast Channel
----------------------------------------------------------------------
1. 1.000000 1045 Home Improvement Show
2. 0.069551 2009 K-12 Online Conference Audio Podcast Channel
3. 0.067529 #SmartBrownVoices - Learning from Diversity
4. -0.020729 #NerdyCast
